In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

Init Plugin
Init Graph Optimizer
Init Kernel


# Fake Dataset

Here is a simple code to generate a fake dataset.

In [2]:
def linear_regression_dataset():
    # --- Fake dataset ---

    np.random.seed(0)

    ntrain = 1000
    nvalid = 100
    ntest = 100

    # number of features
    nc = 4
    # number of outputs
    no = 2

    A = np.random.randn(nc, no)
    b = np.random.randn(no)

    print("True A\n", A)
    print("True b\n", b)

    def generate_one_dataset(A, b, n):
        nc, no = A.shape
        X = np.random.randn(n, nc)
        Y = X.dot(A)+b
        return np.array(X, dtype='float32'), np.array(Y, dtype='float32')

    Xtrain, Ytrain = generate_one_dataset(A, b, ntrain)
    Xvalid, Yvalid = generate_one_dataset(A, b, nvalid)
    Xtest, Ytest = generate_one_dataset(A, b, ntest)

    return (Xtrain, Ytrain), (Xvalid, Yvalid), (Xtest, Ytest)


# Linear Regression

Observe and run the following code.
It performs a linear regression over the artificial dataset.

In [3]:
class LinearRegression(tf.Module):
    def __init__(self, nc, no, name=None):
        super().__init__(name=name)
        # Linear regression parameters
        self.A = tf.Variable(tf.zeros([nc, no]))
        self.b = tf.Variable(tf.zeros([no]))

    @tf.function
    def __call__(self,x):
        return tf.matmul(x, self.A) + self.b


def linear_regression_v1(trainset, validset, testset):

    Xtrain, Ytrain = trainset
    Xvalid, Yvalid = validset
    Xtest, Ytest = testset
    
    # Hyperparameters
    learning_rate = 5e-2
    training_epochs = 100

    # Xtrain (nxtrain,nc)
    nxtrain, nc = Xtrain.shape
    nxtest, _ = Xtest.shape
    # Ytrain (nxtrain,no)
    no = Ytrain.shape[1]

    module = LinearRegression(nc,no)
    
    # We will use the mean square error as loss
    def loss(target,pred):
        return tf.math.reduce_mean(tf.math.squared_difference(target, pred))
        
    # We will use the mean absolute error as metric
    def metric(target,pred):
        return tf.math.reduce_mean(tf.math.abs(target-pred))
    
    # Beware loss and metric are different think.
    # Loss is the differentiable function used for training
    # Metric is a possibly a non-differentiable function use to measure the accuracy.
    
    # NEW ! Optimizer based on gradient descent
    # It computes gradients of loss over var_list items
    # and construct the updates for var_list items.
    # We don't have to write the gradient step by ourselves
    optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

    # We cycle on epochs
    for epoch in range(training_epochs):
        with tf.GradientTape() as tape:
            trainpred = module(Xtrain)
            trainloss = loss(Ytrain,trainpred)
        grads = tape.gradient(trainloss,module.trainable_variables)
        trainmetric = metric(Ytrain,trainpred)
        
        validpred = module(Xvalid)
        validmetric = metric(Yvalid,validpred)
        
        # NEW ! Optimizer based on gradient descent
        # We let the optimizer do the gradient descent
        optimizer.apply_gradients(zip(grads, module.trainable_variables))
        
        print("Epoch %03d\t train loss %f\t train metric %f\t valid metric %f" %
                  (epoch, trainloss.numpy(), trainmetric.numpy(), validmetric.numpy()))

    testpred = module(Xtest)
    testmetric = metric(Ytest,testpred)
    print("Test metric %f"%(testmetric.numpy(),))
        
    # Found parameters
    print("Estimated A\n", module.A.numpy())
    print('Estimated b\n', module.b.numpy())

linear_regression_v1(*linear_regression_dataset())

True A
 [[ 1.76405235  0.40015721]
 [ 0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721]]
True b
 [-0.10321885  0.4105985 ]
Metal device set to: Apple M1


2021-09-23 11:22:42.834224: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-09-23 11:22:42.834369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-09-23 11:22:42.928712: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-23 11:22:42.929147: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-09-23 11:22:42.929279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-09-23 11:22:43.170204: W tensorflow/python/util/ut

Epoch 000	 train loss 7.184082	 train metric 2.142293	 valid metric 2.241359
Epoch 001	 train loss 6.501032	 train metric 2.037724	 valid metric 2.131973
Epoch 002	 train loss 5.883164	 train metric 1.938288	 valid metric 2.028012
Epoch 003	 train loss 5.324233	 train metric 1.843725	 valid metric 1.929139
Epoch 004	 train loss 4.818598	 train metric 1.753836	 valid metric 1.835133
Epoch 005	 train loss 4.361157	 train metric 1.668375	 valid metric 1.745760
Epoch 006	 train loss 3.947302	 train metric 1.587111	 valid metric 1.660756
Epoch 007	 train loss 3.572863	 train metric 1.509852	 valid metric 1.579916
Epoch 008	 train loss 3.234073	 train metric 1.436387	 valid metric 1.503120
Epoch 009	 train loss 2.927526	 train metric 1.366513	 valid metric 1.430090
Epoch 010	 train loss 2.650141	 train metric 1.300064	 valid metric 1.360712
Epoch 011	 train loss 2.399136	 train metric 1.236870	 valid metric 1.294730
Epoch 012	 train loss 2.171991	 train metric 1.176772	 valid metric 1.231959

## Optimization

Yet this previous code work in _eager_ mode which means that the computation is done in interaction with the CPU iteratively. In order to execute the part of the code that is computation costly on an accelerator, we must put that code into `tf.function` which will enable _graph_ mode.

Look at the two follwing codes, they enable that _graph_ mode.
The first is directly inspired by the precedent code.
The second one is using _object oriented programming_ to prevent the use of variables defined outside of a function.
In the following excercices we will prefer this paradigm.


In [4]:
# Direct optimization

class LinearRegression(tf.Module):
    def __init__(self, nc, no, name=None):
        super().__init__(name=name)
        # Linear regression parameters
        self.A = tf.Variable(tf.zeros([nc, no]))
        self.b = tf.Variable(tf.zeros([no]))

    @tf.function
    def __call__(self,x):
        return tf.matmul(x, self.A) + self.b


def linear_regression_v1_acc(trainset, validset, testset):

    Xtrain, Ytrain = trainset
    Xvalid, Yvalid = validset
    Xtest, Ytest = testset
    
    # Hyperparameters
    learning_rate = 5e-2
    training_epochs = 100

    # Xtrain (nxtrain,nc)
    nxtrain, nc = Xtrain.shape
    nxtest, _ = Xtest.shape
    # Ytrain (nxtrain,no)
    no = Ytrain.shape[1]

    module = LinearRegression(nc,no)
    
    # We will use the mean square error as loss
    def loss(target,pred):
        return tf.math.reduce_mean(tf.math.squared_difference(target, pred))
        
    # We will use the mean absolute error as metric
    def metric(target,pred):
        return tf.math.reduce_mean(tf.math.abs(target-pred))
    
    # Beware loss and metric are different think.
    # Loss is the differentiable function used for training
    # Metric is a possibly a non-differentiable function use to measure the accuracy.
    
    # Optimizer based on gradient descent
    # It computes gradients of loss over var_list items
    # and construct the updates for var_list items.
    # We don't have to write the gradient step by ourselves
    optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
    
    # We use a separate train_step function that can be autographed
    @tf.function
    def train_step(x,y,xval, yval):
        with tf.GradientTape() as tape:
            trainpred = module(x)
            trainloss = loss(y,trainpred)
        grads = tape.gradient(trainloss,module.trainable_variables)
        trainmetric = metric(y,trainpred)
        
        validpred = module(xval)
        validmetric = metric(yval,validpred)
        
        # Optimizer based on gradient descent
        # We let the optimizer do the gradient descent
        optimizer.apply_gradients(zip(grads, module.trainable_variables))
        return trainloss, trainmetric, validmetric
    
    # As well, we use a separate test_step function that can be autographed
    @tf.function
    def test_step(x,y):
        testpred = module(x)
        testmetric = metric(y,testpred)
        return testmetric

    # We cycle on epochs
    for epoch in range(training_epochs):
        trainloss, trainmetric, validmetric = train_step(Xtrain,Ytrain, Xvalid, Yvalid)
        
        print("Epoch %03d\t train loss %f\t train metric %f\t valid metric %f" %
                  (epoch, trainloss.numpy(), trainmetric.numpy(), validmetric.numpy()))

    testmetric = test_step(Xtest,Ytest)
    print("Test metric %f"%(testmetric.numpy(),))
        
    # Found parameters
    print("Estimated A\n", module.A.numpy())
    print('Estimated b\n', module.b.numpy())

linear_regression_v1_acc(*linear_regression_dataset())

True A
 [[ 1.76405235  0.40015721]
 [ 0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721]]
True b
 [-0.10321885  0.4105985 ]
Epoch 000	 train loss 7.184082	 train metric 2.142293	 valid metric 2.241359
Epoch 001	 train loss 6.501032	 train metric 2.037724	 valid metric 2.131973
Epoch 002	 train loss 5.883164	 train metric 1.938288	 valid metric 2.028012
Epoch 003	 train loss 5.324233	 train metric 1.843725	 valid metric 1.929139
Epoch 004	 train loss 4.818598	 train metric 1.753836	 valid metric 1.835133
Epoch 005	 train loss 4.361157	 train metric 1.668375	 valid metric 1.745760
Epoch 006	 train loss 3.947302	 train metric 1.587111	 valid metric 1.660756
Epoch 007	 train loss 3.572863	 train metric 1.509852	 valid metric 1.579916
Epoch 008	 train loss 3.234073	 train metric 1.436387	 valid metric 1.503120
Epoch 009	 train loss 2.927526	 train metric 1.366513	 valid metric 1.430090
Epoch 010	 train loss 2.650141	 train metric 1.300064	 valid metric 1.360712
Ep

2021-09-23 11:22:44.336701: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 012	 train loss 2.171991	 train metric 1.176772	 valid metric 1.231959
Epoch 013	 train loss 1.966431	 train metric 1.119623	 valid metric 1.172243
Epoch 014	 train loss 1.780396	 train metric 1.065277	 valid metric 1.115433
Epoch 015	 train loss 1.612025	 train metric 1.013589	 valid metric 1.061404
Epoch 016	 train loss 1.459636	 train metric 0.964423	 valid metric 1.010026
Epoch 017	 train loss 1.321705	 train metric 0.917653	 valid metric 0.961144
Epoch 018	 train loss 1.196856	 train metric 0.873166	 valid metric 0.914636
Epoch 019	 train loss 1.083843	 train metric 0.830860	 valid metric 0.870387
Epoch 020	 train loss 0.981541	 train metric 0.790628	 valid metric 0.828287
Epoch 021	 train loss 0.888930	 train metric 0.752366	 valid metric 0.788231
Epoch 022	 train loss 0.805089	 train metric 0.715965	 valid metric 0.750118
Epoch 023	 train loss 0.729184	 train metric 0.681333	 valid metric 0.713856
Epoch 024	 train loss 0.660462	 train metric 0.648386	 valid metric 0.679353

2021-09-23 11:22:44.801015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [5]:
# Optimization inside a Trainer class

class LinearRegression(tf.Module):
    def __init__(self, nc, no, name=None):
        super().__init__(name=name)
        # Linear regression parameters
        self.A = tf.Variable(tf.zeros([nc, no]))
        self.b = tf.Variable(tf.zeros([no]))

    @tf.function
    def __call__(self,x):
        return tf.matmul(x, self.A) + self.b

class LinearRegressionTrainer:
    
    # We will use the MSE loss
    def _loss(self,target,pred):
        return tf.reduce_mean(tf.math.squared_difference(target, pred))

    # We will use the mean absolute error as metric
    def _metric(self,target,pred):
        return tf.math.reduce_mean(tf.math.abs(target-pred))        

    # We use a separate train_step method that can be autographed
    @tf.function
    def _train_step(self,x,y,xval,yval):
        with tf.GradientTape() as tape:
            trainpred = self.module(x)
            trainloss = self._loss(y,trainpred)
        grads = tape.gradient(trainloss,self.module.trainable_variables)
        trainmetric = self._metric(y,trainpred)

        validpred = self.module(xval)
        validmetric = self._metric(yval,validpred)

        # Optimizer based on gradient descent
        # We let the optimizer do the gradient descent
        self.optimizer.apply_gradients(zip(grads, self.module.trainable_variables))
        return trainloss, trainmetric, validmetric

    # As well, we use a separate test_step method that can be autographed
    @tf.function
    def _test_step(self,x,y):
        testpred = self.module(x)
        testmetric = self._metric(y,testpred)
        return testmetric

    def train(self, trainset, validset, testset,
            learning_rate = 5e-2, training_epochs = 100):

        Xtrain, Ytrain = trainset
        Xvalid, Yvalid = validset
        Xtest, Ytest = testset
        
        # Xtrain (nxtrain,nc)
        nxtrain, self.nc = Xtrain.shape
        nxtest, _ = Xtest.shape
        # Ytrain (nxtrain,no)
        self.no = Ytrain.shape[1]
        
        self.module = LinearRegression(self.nc,self.no)
        self.optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
        
        # We cycle on epochs
        for epoch in range(training_epochs):
            trainloss, trainmetric, validmetric = self._train_step(Xtrain,Ytrain,Xvalid, Yvalid)

            print("Epoch %03d\t train loss %f\t train metric %f\t valid metric %f" %
                      (epoch, trainloss.numpy(), trainmetric.numpy(), validmetric.numpy()))

        testmetric = self._test_step(Xtest,Ytest)
        print("Test metric %f"%(testmetric.numpy(),))

        # Found parameters
        print("Estimated A\n", self.module.A.numpy())
        print('Estimated b\n', self.module.b.numpy())

trainer = LinearRegressionTrainer()
trainer.train(*linear_regression_dataset())

2021-09-23 11:22:44.908867: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


True A
 [[ 1.76405235  0.40015721]
 [ 0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721]]
True b
 [-0.10321885  0.4105985 ]
Epoch 000	 train loss 7.184082	 train metric 2.142293	 valid metric 2.241359
Epoch 001	 train loss 6.501032	 train metric 2.037724	 valid metric 2.131973
Epoch 002	 train loss 5.883164	 train metric 1.938288	 valid metric 2.028012
Epoch 003	 train loss 5.324233	 train metric 1.843725	 valid metric 1.929139
Epoch 004	 train loss 4.818598	 train metric 1.753836	 valid metric 1.835133
Epoch 005	 train loss 4.361157	 train metric 1.668375	 valid metric 1.745760
Epoch 006	 train loss 3.947302	 train metric 1.587111	 valid metric 1.660756
Epoch 007	 train loss 3.572863	 train metric 1.509852	 valid metric 1.579916
Epoch 008	 train loss 3.234073	 train metric 1.436387	 valid metric 1.503120
Epoch 009	 train loss 2.927526	 train metric 1.366513	 valid metric 1.430090
Epoch 010	 train loss 2.650141	 train metric 1.300064	 valid metric 1.360712
Ep

2021-09-23 11:22:45.356739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Dataset

In the precedent example, all the dataset is used at each gradient step.
In most of the case, you can't do that as it will not fit into the memory of the computation engine.
You have to split your data into little batches. Tensorflow dataset are here to help you in that process.

Let's observe the next script.
It presents how to create a Tensorflow dataset object and how to create bacthes to iterate over it.
For each batch, the content of the bacth is displayed as well as the computation of a phony operation.



In [6]:
def play_with_tensorflow_dataset_v1():
    # create fake data

    nsamples = 105

    X = np.arange(nsamples, dtype='float32').reshape(nsamples, 1)
    Y = np.zeros((nsamples, 1), dtype='float32')
    Y[:int(np.ceil(nsamples/2))]=1.
    
    # loop / batch parameters
    nepochs = 5
    batchsize = 8
    
    # Create a dataset from numpy array
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    # Uncomment following line if you want to automatically shuffe the dataset
    #dataset = dataset.shuffle(buffer_size=1000)
    # Each iteration over the dataset will provide batchsize samples
    dataset = dataset.batch(batchsize)
    # or if you want to drop the last non complete batch
    #dataset = dataset.batch(batchsize, drop_remainder=True)

    # Loop over epochs
    for epoch in range(nepochs):
        print('Epoch %d'%(epoch,))
        # Loop over batches
        for batch, (xbatch, ybatch)  in enumerate(dataset):
            print("Epoch %d Batch %d"%(epoch,batch),"\nx", xbatch,"\ny",ybatch)


play_with_tensorflow_dataset_v1()

Epoch 0
Epoch 0 Batch 0 
x tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Batch 1 
x tf.Tensor(
[[ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]
 [13.]
 [14.]
 [15.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Batch 2 
x tf.Tensor(
[[16.]
 [17.]
 [18.]
 [19.]
 [20.]
 [21.]
 [22.]
 [23.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Batch 3 
x tf.Tensor(
[[24.]
 [25.]
 [26.]
 [27.]
 [28.]
 [29.]
 [30.]
 [31.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Batch 4 
x tf.Tensor(
[[32.]
 [33.]
 [34.]
 [35.]
 [36.]
 [37.]
 [38.]
 [39.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]

## Multiple dataset and metrics

What should we do with multiple dataset ? For example how can we iterate over train and validation ?

Moreover, how can we compute a metrics over batches ?

Take a look at the following code which use `tensorflow.metrics` tools.

Please note that it is a fake example without actual training.


In [7]:
def play_with_tensorflow_dataset_v2():

    # create fake data
    nsamples = 105

    Xtrain = np.arange(nsamples, dtype='float32').reshape(nsamples, 1)
    Ytrain = np.ones((nsamples, 1), dtype='float32')

    Xvalid = np.arange(nsamples, dtype='float32').reshape(nsamples, 1)[::-1]
    Yvalid = -np.ones((nsamples, 1), dtype='float32')

    # loop / batch parameters
    nepochs = 5
    batchsize = 8
    
    # Create a dataset from numpy array
    traindataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
    # Uncomment following line if you want to automatically shuffe the dataset
    #traindataset = traindataset.shuffle(buffer_size=1000)
    # Each iteration over the dataset will provide batchsize samples
    traindataset = traindataset.batch(batchsize)
    # or if you want to drop the last non complete batch
    #traindataset = traindataset.batch(batchsize, drop_remainder=True)
    
    # Create a dataset from numpy array
    validdataset = tf.data.Dataset.from_tensor_slices((Xvalid, Yvalid))
    # Uncomment following line if you want to automatically shuffe the dataset
    #validdataset = validdataset.shuffle(buffer_size=1000)
    # Each iteration over the dataset will provide batchsize samples
    validdataset = validdataset.batch(batchsize)
    # or if you want to drop the last non complete batch
    #validdataset = validdataset.batch(batchsize, drop_remainder=True)
    
    # We instanciate a metric for the train and another for the validation
    trainmse = tf.metrics.MeanAbsoluteError()
    validmse = tf.metrics.MeanAbsoluteError()
    
    # Loop over epochs
    for epoch in range(nepochs):
        
        # At the beggining of each epoch, metrics must be reset:
        trainmse.reset_state()
        # Loop over train batches
        for batch, (xbatch, ybatch)  in enumerate(traindataset):
            print("Epoch %d Train Bacth %d"%(epoch,batch),"\nx", xbatch,"\ny",ybatch)
            ypred= xbatch
            #the new batch result is sent to the metric
            trainmse.update_state(ybatch,ypred)
        # At the end we can retrieve the metrics over all the batches
        print("Epoch %d Train Metric %f"%(epoch,trainmse.result()))
        
        # At the beggining of each epoch, metrics must be reset:
        validmse.reset_state()    
        # Loop over valid batches
        for batch, (xbatch, ybatch)  in enumerate(validdataset):
            print("Epoch %d Valid Bacth %d"%(epoch,batch),"\nx", xbatch,"\ny",ybatch)
            ypred= xbatch
            #the new batch result is sent to the metric
            validmse.update_state(ybatch,ypred)
        # At the end we can retrieve the metrics over all the batches
        print("Epoch %d Valid Metric %f"%(epoch,validmse.result()))
                                  
play_with_tensorflow_dataset_v2()

Epoch 0 Train Bacth 0 
x tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Train Bacth 1 
x tf.Tensor(
[[ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]
 [13.]
 [14.]
 [15.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Train Bacth 2 
x tf.Tensor(
[[16.]
 [17.]
 [18.]
 [19.]
 [20.]
 [21.]
 [22.]
 [23.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Train Bacth 3 
x tf.Tensor(
[[24.]
 [25.]
 [26.]
 [27.]
 [28.]
 [29.]
 [30.]
 [31.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Epoch 0 Train Bacth 4 
x tf.Tensor(
[[32.]
 [33.]
 [34.]
 [35.]
 [36.]
 [37.]
 [38.]
 [39.]], shape=(8, 1), dtype=float32) 
y tf.Tensor(
[[1.]
 

## Your turn !

Modify the initial linear regression function to introduce mini batches.
